## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
players = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
players

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,0.367,0.2,1.1,0.217,1.3,3.0,0.423,0.396,0.5,0.6,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,0.339,1.4,4.4,0.325,0.9,2.5,0.363,0.442,1.2,1.4,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,0.529,0.0,0.1,0.100,3.2,5.8,0.539,0.530,2.4,3.2,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [3]:
# answer goes here
filtered_players = players.loc[
                               (players.GS > 20) |
                               ((players.MP/ players.G) > 10)
]
filtered_players

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,0.414,1.5,4.3,0.344,3.4,7.6,0.454,0.477,2.8,3.2,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,0.408,1.6,4.8,0.344,2.6,5.6,0.464,0.488,5.3,6.3,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,0.452,1.9,4.7,0.402,2.3,4.6,0.504,0.554,1.5,1.8,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [4]:
# answer goes here
#ID_players = filtered_players.copy()
#filtered_players['IndexID'] = filtered_players.index
X = filtered_players.loc[
                         :, 
                         ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']
                         ]
X

,PTS,TRB,AST,STL,BLK,TOV
3,13.9,9.5,1.6,1.5,1.0,1.7
4,8.9,7.3,2.2,0.9,0.8,1.5
7,21.3,9.2,2.4,0.5,1.3,1.8
10,10.9,8.4,1.4,0.5,1.5,1.3
12,9.4,7.5,1.3,0.8,0.4,0.9
...,...,...,...,...,...,...
2122,14.1,2.9,5.8,0.9,0.2,2.3
2125,15.3,2.5,2.5,0.9,0.3,1.6
2126,11.7,6.4,1.4,0.7,1.0,0.8
2138,15.1,9.0,1.8,1.5,0.5,1.9


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [5]:
# answer goes here
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_scaled = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
X_scaled

,PTS,TRB,AST,STL,BLK,TOV
0,0.123259,1.611096,-0.607380,1.376834,0.750136,0.030801
1,-0.732937,0.771590,-0.320787,-0.042181,0.377475,-0.208828
2,1.390428,1.496618,-0.225256,-0.988191,1.309129,0.150616
3,-0.390459,1.191343,-0.702911,-0.988191,1.681790,-0.448457
4,-0.647318,0.847908,-0.750677,-0.278683,-0.367849,-0.927715
...,...,...,...,...,...,...
808,0.157506,-0.907424,1.398772,-0.042181,-0.740510,0.749688
809,0.362993,-1.060061,-0.177490,-0.042181,-0.554179,-0.089013
810,-0.253468,0.428155,-0.702911,-0.515186,0.750136,-1.047530
811,0.328746,1.420299,-0.511849,1.376834,-0.181518,0.270430


Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [6]:
kmeans = KMeans(n_clusters=3)

kmeans.fit_predict(X_scaled)

centers = ss.inverse_transform(kmeans.cluster_centers_)
centers = pd.DataFrame(centers, columns = X_scaled.columns)
centers.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,13.954913,8.713873,1.936416,0.779769,1.282081,1.694798
1,10.438829,3.982430,2.033406,0.800217,0.383731,1.230369
2,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765


Experiment with different values of K. Do any further interesting archetypes come out?

For clusters=3, we seem to have clustered the players into offensive and defensive players. As we increase K, we see that the maximum statistical values per variable continue to increase. 

This is because we are creating a gradient of the performance metrics. The more clusters we define, the more the top performing clusters appear to perform.

In [8]:
# answer goes here
kmeans = KMeans(n_clusters=6)

kmeans.fit_predict(X_scaled)

centers = ss.inverse_transform(kmeans.cluster_centers_)
centers = pd.DataFrame(centers, columns = X_scaled.columns)
centers.style.background_gradient()




,PTS,TRB,AST,STL,BLK,TOV
0,7.649010,3.501980,1.368317,0.604455,0.365347,0.909406
1,23.429268,7.768293,7.482927,1.673171,0.807317,3.700000
2,11.861006,7.377987,1.742767,0.678616,0.948428,1.424528
3,17.467925,10.800000,2.150943,0.905660,1.884906,2.150943
4,18.362319,4.536232,5.509420,1.230435,0.385507,2.531884
5,13.018636,4.061818,2.727273,1.044545,0.340455,1.526818


In [9]:
kmeans = KMeans(n_clusters=20)

kmeans.fit_predict(X_scaled)

centers = ss.inverse_transform(kmeans.cluster_centers_)
centers = pd.DataFrame(centers, columns = X_scaled.columns)
centers.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,11.500000,2.630769,4.641026,0.741026,0.141026,1.900000
1,8.523810,5.055556,1.422222,0.863492,0.606349,0.974603
2,15.325000,9.030556,1.861111,0.622222,0.688889,1.688889
3,14.725000,8.191667,3.102778,1.038889,1.194444,1.969444
4,18.832609,4.128261,3.117391,0.908696,0.363043,1.960870
5,26.160000,7.200000,9.473333,1.666667,0.633333,4.433333
6,8.881967,6.737705,1.322951,0.518033,1.049180,1.222951
7,14.620833,12.766667,1.516667,0.841667,1.812500,1.912500
8,22.951111,4.708889,5.700000,1.088889,0.346667,2.813333
9,5.896809,3.225532,1.014894,0.487234,0.328723,0.722340
